In [114]:
%pip install python-Levenshtein
%pip install openpyxl
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [115]:
Month="May"

In [116]:
import warnings
# Disable only DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Disable all warnings
warnings.filterwarnings("ignore")
import Levenshtein
import pandas as pd
def levenshtein_distance(str1, str2):
    """
    Calculate the Levenshtein distance between two strings.

    Parameters:
    str1 (str): The first string.
    str2 (str): The second string.

    Returns:
    int: The Levenshtein distance between the two strings.
    """
    return Levenshtein.distance(str1, str2)
def updateEmployeeCode(dfMetric):
    dfEmployeeCode=pd.read_csv('report model\\EmployeeCode.csv')
    dfMetric=dfMetric.rename(columns={'userName':'Name','TimeInHours':'CompletedWork'})
    dfMetric.head()
    dfMetric['Employee ID']=""
    dfMetric['Name TFS']=""
    for i,row in dfMetric.iterrows():
        index=-1
        distance=1000000
        for j ,row2 in dfEmployeeCode.iterrows():
            
            distance1=levenshtein_distance(row['Name'], row2['Name'])
            if distance1<distance:
                    distance=distance1
                    index=j
        #print (row['Name'],dfTFS.iloc[index]['Name'])
        dfMetric.at[i,'Employee ID']=dfEmployeeCode.iloc[index]['Employee ID']
        dfMetric.at[i,'Name TFS']=dfEmployeeCode.iloc[index]['Name']   
    dfMetric=dfMetric.drop(columns={'Name'})
    dfMetric=dfMetric.rename(columns={'Name TFS':'Name'})
    return dfMetric


def UpdateEpicName(df,dfsource):
    for index, row in df.iterrows():
        dfresult=dfsource[dfsource['Budget Code']==row['Budget Code']]
        if len(dfresult)!=0:
            df.at[index,'Epic Name']=dfresult.iloc[0]['Product']
        else:
            print (row['Budget Code'] ,row['Epic Name'])
    return df
def update_budget_codes(df_codes, df_data, code_col='Budget Code'):
    """
    Updates budget codes in df_data based on a mapping provided in df_codes.

    Parameters:
        df_codes (pd.DataFrame): DataFrame containing 'old_budget_code' and 'new_budget_code' columns.
        df_data (pd.DataFrame): DataFrame where budget codes should be updated.
        code_col (str): Column name in df_data that contains budget codes.

    Returns:
        pd.DataFrame: A new DataFrame with updated budget codes.
    """

    # Create a dictionary mapping old codes to new codes
    code_map = df_codes.set_index('Budget Code')['New Budget code'].to_dict()

    # Debug print (optional): Check the mapping
    print("Budget code mapping:", code_map)

    # Replace the budget codes in df_data using the map
    df_data_updated = df_data.copy()
    df_data_updated[code_col] = df_data_updated[code_col].replace(code_map)

    return df_data_updated

def UpdateTeam(df,dfsource):
    for index, row in df.iterrows():
        dfresult=dfsource[dfsource['Employee ID']==row['Employee ID']]
        if len(dfresult):
            df.at[index,'Team']=dfresult.iloc[0]['Team']
    return df

def UpdatePortfolio(df,dfsource):
    
    print (print(df.dtypes))
    print (print(dfsource.dtypes))
    
    df['Portfolio']=""
    for index, row in df.iterrows():
        dfresultP=dfsource[dfsource['Budget Code']==row['Budget Code']]
        if len(dfresultP):
            if row['Employee ID']=='311032':
                print (dfresultP.iloc[0]['Portfolio'])
            df.iloc[index,6]=dfresultP.iloc[0]['Portfolio']
        else:
            print ("No row",row['Budget Code'],row['Employee ID'])
    return df

#Select emtpy Budget code
def ProcessData(df):
    dfTFSEmpty=df[df['Budget Code'].isna()]
    dfTFSEmpty["TotalHours"]=0
    dfTFSEmpty["Percentage"]=0

    #Calculate the percentage 

    # Group by EmployeeID and Project, then sum the hours
    grouped = df.groupby(['Budget Code','Epic Name','Employee ID','Name'])['CompletedWork'].sum().reset_index()

    # Calculate the total hours worked by each employee
    total_hours = grouped.groupby('Employee ID')['CompletedWork'].sum().reset_index()
    total_hours = total_hours.rename(columns={'CompletedWork': 'TotalHours'})

    # Merge the total hours with the grouped data
    merged = pd.merge(grouped, total_hours, on='Employee ID')

    # Calculate the percentage of time invested in each project
    merged['Percentage'] = (merged['CompletedWork'] / merged['TotalHours']) * 100

    print (merged.count())
    merged.head()

    for i, row in  dfTFSEmpty.iterrows():
        employeedata= merged[merged['Employee ID']==row['Employee ID']]
        max_completed_work_idx = employeedata.groupby('Employee ID')['CompletedWork'].idxmax()
        df_max_completed_work = employeedata.loc[max_completed_work_idx]
        if len(df_max_completed_work)>0:
            #print (df_max_completed_work.iloc[0]['Budget Code'])
            dfTFSEmpty.at[i,'Budget Code']=df_max_completed_work.iloc[0]['Budget Code']
            dfTFSEmpty.at[i,'Epic Name']=df_max_completed_work.iloc[0]['Epic Name']
        #else:
            #print (row['Employee ID'])

    final_result = pd.concat([dfTFSEmpty, merged])
    final_result.reset_index(drop=True, inplace=True)
    return final_result


In [117]:

# Example usage
string1 = "Mehdi CHAIB RASSOU"
string2 = "Mahdi Rassou"
#string2 = "Mahdi Chebbah"

distance = levenshtein_distance(string1, string2)
print(f"The Levenshtein distance between '{string1}' and '{string2}' is {distance}.")

The Levenshtein distance between 'Mehdi CHAIB RASSOU' and 'Mahdi Rassou' is 12.


In [118]:
# load support data
dfOldResult=pd.read_csv('backup\\oldresult.csv')
dfPorductCode=pd.read_csv('D:\\Medad Projects\\PI planning\\Budget Code\\Portfolio Budget Code_V3.csv')
dfBackup=pd.read_csv('backup\\tsfresult.csv')
dfNewCode=pd.read_excel('D:\\Medad Projects\\PI planning\\Budget Code\\NewBudgetCode.xlsx')


In [119]:
dfNewCode.head()

,Platform Name,Sub module,Budget Code,New Budget code
0,Library Services Platform,NaN,20100,20100
1,NaN,DAR,20313,20314
2,NaN,Library Portal LP,20213,20236
3,Training Management Platform,NaN,10200,10200
4,NaN,Training Environment,10201,10211


In [120]:
dfBackup2=dfBackup[dfBackup['Employee ID']=='FR-KSA-158']
dfBackup2.head()

,Unnamed: 0,Budget Code,Epic Name,Employee ID,Name,CompletedWork,TotalHours,Percentage
209,169,20213,Portal Core,FR-KSA-158,Yasmin Emad,150.0,150.0,80.0


In [121]:
Month

'May'

In [122]:
import os

def create_folder(path):
    """
    Creates a folder at the specified path.

    Parameters:
        path (str): The path where the folder will be created.

    Returns:
        bool: True if the folder was created successfully, False if it already exists.
    """
    try:
        os.makedirs(path, exist_ok=False)
        print(f"Folder created successfully: {path}")
        return True
    except FileExistsError:
        print(f"Folder already exists: {path}")
        return False
    except OSError as e:
        print(f"Error creating folder '{path}': {e}")
        raise

In [123]:
create_folder("Result\\"+Month)

dfTFS=pd.read_excel("Data\\"+Month+"\\Medad TFS.xlsx")
print (dfTFS.count())
dfTFS=ProcessData(dfTFS)
print (dfTFS.head())

# update the budget code from old data 

for index, row in dfTFS.iterrows():
    if pd.isna(row['Budget Code'] ):
       
        result=dfBackup[dfBackup['Employee ID']==row['Employee ID']]
        if len(result):
            print ( result.iloc[0]['Budget Code']  )
            dfTFS.at[index, 'Budget Code']   =  result.iloc[0]['Budget Code']  
            dfTFS.at[index, 'Epic Name']   =  result.iloc[0]['Epic Name']  
            print (index)
        else:
            print (row['Employee ID'])
dfTFS.to_csv("Result\\"+Month+"\\tsfresult.csv")

Folder already exists: Result\May
Budget Code      54
Epic Name        57
Employee ID      67
Name             67
CompletedWork    67
dtype: int64
Budget Code      54
Epic Name        54
Employee ID      54
Name             54
CompletedWork    54
TotalHours       54
Percentage       51
dtype: int64
   Budget Code               Epic Name Employee ID              Name  \
0      20110.0       Enhance Medad LSP      110032  Ahmad Abduldayem   
1      20110.0       Enhance Medad LSP      110503    Samah Mohammed   
2          NaN                     NaN      293002       Yamen Saleh   
3      20309.0  Medad Content Plus V.1  FR-KSA-109     Mohammed Rezk   
4          NaN                     NaN  FR-KSA-113     Mohammed Emad   

   CompletedWork  TotalHours  Percentage  
0           56.0         0.0         0.0  
1          106.0         0.0         0.0  
2          160.0         0.0         0.0  
3            2.0         0.0         0.0  
4            0.5         0.0         0.0  
20313
2
1

In [124]:

# timeDoctor
dftimeDoctor=pd.read_excel("Data\\"+Month+"\\timeDoctor.xlsx")
dftimeDoctor=dftimeDoctor.rename(columns={'userName':'Name','TimeInHour':'CompletedWork'})
# update un budget code 
print (dftimeDoctor.head())

dftimeDoctor=ProcessData(dftimeDoctor)
for index, row in dftimeDoctor.iterrows():
    if pd.isna(row['Budget Code'] ):
        result=dfBackup[dfBackup['Employee ID']==row['Employee ID']]
        if len(result):
            #print ( result.iloc[0]['Budget Code']  )
            dftimeDoctor.at[index, 'Budget Code']   =  result.iloc[0]['Budget Code']  
            dftimeDoctor.at[index, 'Epic Name']   =  result.iloc[0]['Epic Name']  
            #print (index)
        else:
            print ("Not found",row['Employee ID'] )
dftimeDoctor.to_csv("Result\\"+Month+"\\timeDoctorresult.csv")

  Budget Code Epic Name Employee ID             Name  CompletedWork
0         NaN       NaN      101259  Zahra Albensaad          79.96
1         NaN       NaN      101307   Reham Binjaber          29.44
2         NaN       NaN      101388    Ali Almatrood          56.07
3         NaN       NaN      120017  Mohammed Swidan         164.54
4         NaN       NaN      311028   Mira Boureslan          78.92
Budget Code      95
Epic Name        95
Employee ID      95
Name             95
CompletedWork    95
TotalHours       95
Percentage       95
dtype: int64
Not found FR-KSA-114
Not found FR-KSA-163
Not found nan


In [125]:

#filter the data
dfTFS=dfTFS[['Employee ID','Name','CompletedWork','Budget Code','Epic Name','TotalHours','Percentage']]
dftimeDoctor=dftimeDoctor[['Employee ID','Name','CompletedWork','Budget Code','Epic Name','TotalHours','Percentage']]

In [126]:
# update budget code 
dfTFS=UpdateEpicName(dfTFS,dfPorductCode)
dftimeDoctor=UpdateEpicName(dftimeDoctor,dfPorductCode)
dfOldResult=UpdateEpicName(dfOldResult,dfPorductCode)
dfTFS.to_csv("Result\\"+Month+"\\tsfresult.csv")
dftimeDoctor.to_csv("Result\\"+Month+"\\timeDoctorresult.csv")

20313 DAR Core
10413 IEP Core
70100 Medad Unified Core
20313 DAR Core
20110 Enhance Medad LSP
10402 Outcomes Assessment
71100 Integration Management
10301 Student Management
20309 Medad Content Plus V.1
10101 Learning Environment
10401 Core Service
nan nan
10302 Admission
10302 Admission
20313 DAR Core
20213 Portal Core
10302 Admission
70900 Commerce and Billing gateway
10210 Individual Development Plan (IDP)
10101 Learning Environment
20213 Portal Reported & known issues
10101 Learning Environment
10401 Core Service
20213 New Portal Design
20313 DAR Core
20501 HUB Gateway
nan nan
20213 Portal Search
10204 Training Environment
10402 Outcomes Assessment
20309 Medad Content Plus V.1
10210 Individual Development Plan (IDP)
10302 Admission
10401 Core Service
10301 Student Management
20110 Enhance Medad LSP
70300 Services Provisioning & Management
10402 Outcomes Assessment
20110 Enhance Medad LSP
20110 Enhance Medad LSP
70100 Medad Unified Core
20501 Epic: Releases
10402 Outcomes Assessment

In [127]:

dfEmployeeCode=pd.read_csv("report model\\EmployeeCode.csv")
dfCapacity=pd.read_excel("Data\\"+Month+"\\Capacity.xlsx")

dfCapacity['Budget Code']=""
dfCapacity['Epic Name']=""
dfCapacity['CompletedWork']=0
dfCapacity['TotalHours']=0
dfCapacity['Percentage']=0
#dfCapacity=dfCapacity.drop(columns={'Role'})
dfCapacity=dfCapacity.rename(columns={'EmployeeID':'Employee ID'})
print (dfCapacity.head())
for i ,row in dfCapacity.iterrows():
   
    try :
        reminingPart= round ((1-row['TLI%'])*row[2],2)
        dfresult=dftimeDoctor[dftimeDoctor['Employee ID']==row['Employee ID']]
        if len(dfresult)==0:
             dfresult=dfTFS[dfTFS['Employee ID']==row['Employee ID']]
             if len(dfresult)==0:
                dfresult=dfOldResult[dfOldResult['Employee ID']==row['Employee ID']]

        print ("Employee ID:",row['Employee ID'],len(dfresult))

        dfresult['CompletedWork'] = pd.to_numeric(dfresult['CompletedWork'], errors='coerce')
        max_completed_work_idx = dfresult.groupby('Employee ID')['CompletedWork'].idxmax()
        df_max_completed_work = dfresult.loc[max_completed_work_idx]
        if len(df_max_completed_work)>0:
            #print (df_max_completed_work.iloc[0]['Budget Code'])


            dfCapacity.at[i,'Budget Code']=df_max_completed_work.iloc[0]['Budget Code']
            dfCapacity.at[i,'Epic Name']=df_max_completed_work.iloc[0]['Epic Name']
     
            
            dfCapacity.at[i,'Percentage']=round ((1-row['TLI%']),2)
            dfCapacity.at[i,'TLI%']=round ((row['TLI%']),2)
            #dfCapacity.at[i,'Productivity over Availability']=round (row['Productivity over Availability'],2)


            dfCapacity.at[i,'CompletedWork']=reminingPart
            dfCapacity.at[i,'TotalHours']=reminingPart
            dfCapacity.at[i,'Percentage']=round ((1-row['TLI%']),2)
            
        else:
           print (reminingPart,row['Name'])
           dfCapacity.at[i,'CompletedWork']=reminingPart
           dfCapacity.at[i,'TotalHours']=reminingPart
           dfCapacity.at[i,'Percentage']=round ((1-row['TLI%']),2)
    except Exception:
         print (Exception)

for i ,row in dfCapacity.iterrows():            

            if row['TLI%']>=1:
                     
                     reminingPart= round ((1-row['TLI%'])*row[2],2)
                     print (reminingPart)
                     if row['Name']=="Diaa Shalabi":
                         print ("prod:",row['TLI%'])
                         print ("remining time",reminingPart)
                     filtered_result=dfTFS[dfTFS['Employee ID']==row['Employee ID']]
                     if not filtered_result.empty:
                        print (row['Employee ID'])
                        index_of_max = filtered_result['CompletedWork'].idxmax()
                        print (index_of_max)
                        print ("Completed hours:",dfTFS.loc[index_of_max, 'CompletedWork'])
                        # Update the completedWork value at this index
                        timecount =dfTFS.iloc[index_of_max]['CompletedWork'] +reminingPart
                        print (timecount)
                        if timecount <0:
                             dfTFS.loc[index_of_max, 'CompletedWork'] = 0
                             reminingPart=timecount
                             while timecount<0:
                                reminingPart=timecount
                                print ("within loop",reminingPart)
                                filtered_result=dfTFS[dfTFS['Employee ID']==row['Employee ID']]
                                index_of_max = filtered_result['CompletedWork'].idxmax()
                                print ("Index:",index_of_max)
                                if dfTFS.iloc[index_of_max]['CompletedWork'] ==0:
                                     print ("End")
                                     break
                                timecount =dfTFS.iloc[index_of_max]['CompletedWork'] +reminingPart
                                print (reminingPart)
                                if timecount<0:
                                    dfTFS.loc[index_of_max, 'CompletedWork'] = 0
                                else:
                                    dfTFS.loc[index_of_max, 'CompletedWork'] = dfTFS.loc[index_of_max, 'CompletedWork'] +reminingPart


                        else:
                            dfTFS.loc[index_of_max, 'CompletedWork'] = dfTFS.loc[index_of_max, 'CompletedWork'] +reminingPart
            
            
dfCapacity=UpdateTeam(dfCapacity,dfEmployeeCode)
dfCapacityfinal=dfCapacity[dfCapacity['CompletedWork']>0]

dfCapacityfinal=dfCapacityfinal[['Employee ID','Name','Budget Code','Epic Name','CompletedWork','TotalHours','Percentage']]

dfCapacity.to_csv('Result\\'+Month+'\\CapacityResults.csv')
dfCapacityfinal["ReportType"]="Remaining Capacity" 
dfTFS["ReportType"] ="TFS"

dftimeDoctor["ReportType"] ="TimeDoctor"
dfTFS['CompletedWork']=dfTFS['CompletedWork'].round(2)
dffinalResult= pd.concat([dftimeDoctor, dfTFS,dfCapacityfinal])
dffinalResult=dffinalResult[dffinalResult['CompletedWork']>0]
dffinalResult.reset_index(drop=True, inplace=True)
dfCapacityfinal=dfCapacityfinal.drop(columns={'ReportType'})
dfTFS=dfTFS.drop(columns={'ReportType'})
dffinalResult.to_csv("finalResult.csv")
dffinalResult=dffinalResult.drop(columns={'TotalHours','Percentage'})

dffinalResult['Budget Code'] = dffinalResult['Budget Code'].replace('', '0').fillna('0')
# Convert column type from object (string) to integer
dffinalResult['Budget Code'] = dffinalResult['Budget Code'].astype(int)

dffinalResult=UpdatePortfolio(dffinalResult,dfPorductCode)
dffinalResult=update_budget_codes(dfNewCode,dffinalResult)
dffinalResult.to_csv('Result\\'+Month+'\\Total Completed.csv')
name='Result\\'+Month+'\\'+Month+'-Report.xlsx'
dfCapacity=dfCapacity.rename(columns={'TotalHours':'Variance','Percentage':'Variance %'})
dfCapacity=dfCapacity.drop(columns={'CompletedWork'})
dfCapacityfinal=update_budget_codes(dfNewCode,dfCapacityfinal)
dfTFS=update_budget_codes(dfNewCode,dfTFS)
with pd.ExcelWriter(name, engine='openpyxl') as writer:
    dffinalResult.to_excel(writer, sheet_name='Total Completed-'+Month, index=False)
    dfCapacity.to_excel(writer, sheet_name=Month+' Capacity', index=False)
    dfCapacityfinal.to_excel(writer, sheet_name='filtered '+Month+' Capacity', index=False)
    dfTFS.to_excel(writer, sheet_name='TFS time', index=False)
    dftimeDoctor.to_excel(writer, sheet_name='Time Doctor time', index=False)


  Employee ID                      Name  \
0      100363  Mohammed Hussam Alkhatib   
1      100396         Aissa Berrachiche   
2      100766          Ahmad Abdulsalam   
3      100934            Alaa Alsharaah   
4      101031              Mohammed Gad   

   Capacity (Exclude official holidays&weekends)      TLI%  Billable  AUC  \
0                                            168  0.476190       NaN  NaN   
1                                            168  1.377381       NaN  NaN   
2                                            168  0.922619       NaN  NaN   
3                                            168       NaN       NaN  NaN   
4                                            168  0.238095       NaN  NaN   

   Internal  Investment  Medad ERP  Medad TFS  ...  TimeDoctor non Billable  \
0       NaN         NaN        NaN        NaN  ...                      NaN   
1       NaN         NaN        NaN       12.0  ...                      NaN   
2       NaN         NaN        NaN      1